In [ ]:
%%capture
!pip install gdown

In [ ]:
import gdown

import pandas as pd
import matplotlib.pyplot as plt

import xml.etree.ElementTree as ET

In [ ]:
%%capture

ROUTE_GPX = 'route.gpx'
ROUTE_GPX_URL = 'https://drive.google.com/file/d/1-kx84-fNAOuDWSdVEYU_DjP4Om0yHnaD/view?usp=sharing'

gdown.download(ROUTE_GPX_URL, ROUTE_GPX, quiet=True, fuzzy=True)

In [ ]:
def parse_gpx(filepath):
    root = ET.parse(filepath).getroot()

    ns = '{http://www.topografix.com/GPX/1/1}'
    data = {
        'time': [], 'lat': [], 'lon': [], 'baro': []
    }

    for trk in root.findall(f".//{ns}trk"):
        for trkseg in trk.findall(f"{ns}trkseg"):
            for trkpt in trkseg.findall(f"{ns}trkpt"):
                data['lat'].append(float(trkpt.get('lat')))
                data['lon'].append(float(trkpt.get('lon')))

                time_elem = trkpt.find(f"{ns}time")
                data['time'].append(time_elem.text if time_elem is not None else None)

                ext = trkpt.find(f"{ns}extensions")
                if ext:
                    elem = ext.find(f"{ns}baro")
                    data['baro'].append(float(elem.text) if elem is not None else None)

    df = pd.DataFrame(data)
    return df

df = parse_gpx(ROUTE_GPX)
df.head()

In [ ]:
import plotly.graph_objects as go

# Assuming your DataFrame is named 'df' with columns 'lat', 'lot', 'baro'
fig = go.Figure(data=[go.Scatter3d(
    x=df['lat'],
    y=df['lon'],
    z=df['baro'],
    mode='markers',
    marker=dict(
        size=5,
        color=df['baro'],                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)])

# Setting the layout for the plot
fig.update_layout(
    scene = dict(
        xaxis_title='Latitude',
        yaxis_title='Longitude',
        zaxis_title='Barometric Pressure'
    ),
    width=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

# Display the plot
fig.show()